# Week 3
## Monday June 3, 2019:
To start out today, I decided to make a smaller .csv file to test the python canopy clustering code. I used Multigenome_CSV_Generator.py to generate one large .csv file of counts of the top 10 kmers from the previously generated .csv files from 10 genomes. I then tried to run my python code for canopy clustering. While running, the .csv file stored in a data frame used about 16 GB of RAM, and the CPU load of my mac grew sharply when trying to calculate the distance matrix. I was able to get lots of small clusters since the cluster cut-off values were lower than the distance values from the Euclidean metric. I then switched the distance metric to cosine similarity (range -1 to 1) and tried to find a good cut-off value for canopy inclusion. Once I had good clustering, I tried to generate a list of groupings to import into FindMyFriends using the manualGrouping() method. For this, I generated a list where the list index was a protein, and the value of the list at that index was the grouping of the protein. Once I examined the grouping list, I then realized that the code had an error; points were being clustered multiple times. I fixed that issue and finalized the code:

In [ ]:
# modified from gdbasset canopy.py gist on github: https://gist.github.com/gdbassett/528d816d035f2deaaca1

import numpy as np
import pandas as pd
import csv
from sklearn.metrics.pairwise import pairwise_distances

# X shoudl be a numpy matrix, very likely sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
# T1 > T2 for overlapping clusters
# T1 = Distance to centroid point to not include in other clusters
# T2 = Distance to centroid point to include in cluster
# T1 > T2 for overlapping clusters
# T1 < T2 will have points which reside in no clusters
# T1 == T2 will cause all points to reside in mutually exclusive clusters
# Distance metric can be any from here: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html
# filemap may be a list of point names in their order in X. If included, row numbers from X will be replaced with names from filemap. 
 
def canopy(X1_dist, T1, T2, filemap=None):
    canopies = dict()
    canopy_points = set(range(X1_dist.shape[0]))
    grouping_list = [None] * X1_dist.shape[0]
    iteration = 0
    clustered_points = []
    while canopy_points:
        iteration = iteration + 1
        print("iteration: " + str(iteration))
        point = canopy_points.pop()
        i = len(canopies)
        points_in_T2 = np.where(X1_dist[point] < T2)[0]
        points_to_cluster = set(points_in_T2).difference(set(clustered_points))
        canopies[i] = {"c":point, "points": points_to_cluster}
        clustered_points.extend(points_to_cluster)
        canopy_points = canopy_points.difference(set(np.where(X1_dist[point] < T1)[0]))
        
        for entry in canopies[i]["points"]:
            grouping_list[entry] = point
        
    if filemap:
        for canopy_id in canopies.keys():
            canopy = canopies.pop(canopy_id)
            canopy2 = {"c":filemap[canopy['c']], "points":list()}
            for point in canopy['points']:
                canopy2["points"].append(filemap[point])
            canopies[canopy_id] = canopy2
    return grouping_list

def main():
    df = pd.read_csv('/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA_files/10_genome_k4_kmer_top_10_table.csv')
    #gets protein_ids
    kmers = list(df['Unnamed: 0'])
    df['kmers'] = kmers
    #move from data column to index column
    df.set_index('kmers',inplace = True)
    #delete protein header list
    del df['Unnamed: 0']
    df = df.T
    df.head(3)
    print("canopy clustering:")
    X1_dist = pairwise_distances(df.to_numpy(), metric='cosine')
    
    grouping_list = canopy(X1_dist, 0.99, 0.99)
    
    '''
    with open('cosine_canopy_clusters_0.99.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in clusters.items():
            writer.writerow([key, value])
    '''
    with open('grouping_list.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(grouping_list) 
    
if __name__ == "__main__":
    main()

## Tuesday June 4, 2019:
To recap, what I have so far:
* download genomes from GenBank using Batch Entrez 
* use prodigal to convert .fasta files to .faa files 

In [ ]:
prodigal -i inputfilename.fna -a outputfilename.faa

* generate .csv files of amino acid kmer counts (using Dayhoff compressed alphabet) for each genome (kmer_counter_test1.R)


In [ ]:
library("ape")
library("kmer")
library("MASS")
setwd("/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA_files")


file_list <- c("GCA_000005845.2_ASM584v2","GCA_000006665.1_ASM666v1","GCA_000006925.2_ASM692v2","GCA_000007445.1_ASM744v1","GCA_000008865.2_ASM886v2","GCA_000010245.1_ASM1024v1","GCA_000010485.1_ASM1048v1","GCA_000012005.1_ASM1200v1","GCA_000012025.1_ASM1202v1","GCA_000019645.1_ASM1964v1","GCA_000026325.2_ASM2632v2","GCA_000091005.1_ASM9100v1","GCA_000092525.1_ASM9252v1","GCA_000167875.1_ASM16787v1","GCA_000176575.2_ASM17657v2","GCA_000176655.2_ASM17665v2","GCA_000176695.2_ASM17669v2","GCA_000333215.1_ASM33321v1","GCA_000350785.1_Esch_coli_KTE21_V1","GCA_000613265.1_ASM61326v1","GCA_000690815.1_ASM69081v1","GCA_000714595.1_ASM71459v1","GCA_000734955.1_ASM73495v1","GCA_002007705.1_ASM200770v1","GCA_003697165.2","GCA_900092615.1_PRJEB14041")

for (file in file_list)
{
  file_path <- paste(file,'.faa',sep = '')
  out_path <- paste(file,'_kmer_k5_matrix.csv',sep = '')
  proteins <- read.FASTA(file_path, type = "AA")
  print(paste('Genome ',file,' proteins are loaded'))
  kmerCounts <- kcount(proteins, k = 5)
  print(paste('Genome ',file, ' kmerCounts is finished'))
  write.csv(as.matrix(kmerCounts), file = out_path)
  print(paste('Genome ', file, ' matrix is output'))
  print(paste('Genome ',file,' is complete', sep = ''))
}

* combine separate .csv files into one large .csv file for each genome (Multigenome_CSV_Generator.py)

In [ ]:
### CSV File Generator:
import glob, pandas as pd
data_folder = '/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA_files/10_genomes_4mer_csv/'
file_list = glob.glob(data_folder + '*kmer_k4_matrix*')
dicty = dict()
for file in file_list:

    df = pd.read_csv(file)
    protein_id = list(df['Unnamed: 0'])
    protein_id = [x.split(' ')[0] for x in protein_id]
    df['protein_id'] = protein_id
    df.set_index('protein_id',inplace = True)
    del df['Unnamed: 0']
    
    df = df.T
    
    for col in list(df.columns):
        loop = df[df[col] != 0]
        loop = loop[col]
        loop = loop.sort_values(ascending = False)
        loop = loop[0:10]
        dicty[col] = loop

df_test = pd.DataFrame(dicty)

df2 = df_test.fillna(0)
df2.to_csv('/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA_files/10_genome_k4_kmer_top_10_table.csv')

* read in combined .csv file and cluster genes based on cosine similarity of kmer profiles (canopyClusteringTest.py)

In [ ]:
import numpy as np
import pandas as pd
import csv
from sklearn.metrics.pairwise import pairwise_distances

# X shoudl be a numpy matrix, very likely sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
# T1 > T2 for overlapping clusters
# T1 = Distance to centroid point to not include in other clusters
# T2 = Distance to centroid point to include in cluster
# T1 > T2 for overlapping clusters
# T1 < T2 will have points which reside in no clusters
# T1 == T2 will cause all points to reside in mutually exclusive clusters
# Distance metric can be any from here: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html
# filemap may be a list of point names in their order in X. If included, row numbers from X will be replaced with names from filemap. 
 
def canopy(X1_dist, T1, T2, filemap=None):
    canopies = dict()
    canopy_points = set(range(X1_dist.shape[0]))
    grouping_list = [None] * X1_dist.shape[0]
    iteration = 0
    clustered_points = []
    while canopy_points:
        iteration = iteration + 1
        print("iteration: " + str(iteration))
        point = canopy_points.pop()
        i = len(canopies)
        points_in_T2 = np.where(X1_dist[point] < T2)[0]
        points_to_cluster = set(points_in_T2).difference(set(clustered_points))
        canopies[i] = {"c":point, "points": points_to_cluster}
        clustered_points.extend(points_to_cluster)
        canopy_points = canopy_points.difference(set(np.where(X1_dist[point] < T1)[0]))
        
        for entry in canopies[i]["points"]:
            grouping_list[entry] = point
        
    if filemap:
        for canopy_id in canopies.keys():
            canopy = canopies.pop(canopy_id)
            canopy2 = {"c":filemap[canopy['c']], "points":list()}
            for point in canopy['points']:
                canopy2["points"].append(filemap[point])
            canopies[canopy_id] = canopy2
    return grouping_list

def main():
    df = pd.read_csv('/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA_files/10_genome_k4_kmer_top_10_table.csv')
    #gets protein_ids
    kmers = list(df['Unnamed: 0'])
    df['kmers'] = kmers
    #move from data column to index column
    df.set_index('kmers',inplace = True)
    #delete protein header list
    del df['Unnamed: 0']
    df = df.T
    df.head(3)
    print("canopy clustering:")
    X1_dist = pairwise_distances(df.to_numpy(), metric='cosine')
    
    grouping_list = canopy(X1_dist, 0.99, 0.99)
    
    '''
    with open('cosine_canopy_clusters_0.99.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in clusters.items():
            writer.writerow([key, value])
    '''
    with open('grouping_list.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(grouping_list) 
    
if __name__ == "__main__":
    main()

Next, I was able to import the groupings from the .csv file into R and use them as the groupings of a pangenome object:

In [ ]:
library(FindMyFriends)
library(kebabs)

setwd("/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/FAA_files/")
genomeFiles <- list.files(getwd(), full.names=TRUE, pattern='*.faa')
pan <- pangenome(genomeFiles[1:10], translated=TRUE, geneLocation='prodigal', lowMem=FALSE)
groupingList <- as.integer(read.csv("grouping_list.csv", header=FALSE, sep=',', stringsAsFactors = FALSE))
pan <- manualGrouping(pan, groupingList)
plotStat(pan, type='qual', palette = 6)
plotSimilarity(pan)

Because I chose such a high threshold for the clustering step, I only had a small amount of groups for analysis. I was able to generate graphs, but I need to see how accurate they are. ![summ](images/w3/ManGroupSumm_0.99.png) ![sim](images/w3/ManGroupSim_0.99.png)

From what I can tell by the groupings of the genomes that Kaleb sent, this grouping seems too coarse. Last week's heatmap grouped similar phylotypes together, but they are all jumbled in this heatmap. It is likely due to the clustering threshold, so I will have to tweak that later. This just serves as a great proof of concept that we can pre-cluster the data and then successfully get it into FindMyFriends!

## Wednesday June 5, 2019:
My first goal today is to modify my clustering algorithm to separate out paralogues (duplicate genes in the same genome) into different groups. We are aiming for approximately one group per protein, and each orthologous protein from each strain should hopefully end up in the same cluster. My code: 

In [ ]:
import pandas as pd
import csv
from sklearn.metrics.pairwise import pairwise_distances

# X shoudl be a numpy matrix, very likely sparse matrix: http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix
# T1 > T2 for overlapping clusters
# T1 = Distance to centroid point to not include in other clusters
# T2 = Distance to centroid point to include in cluster
# T1 > T2 for overlapping clusters
# T1 < T2 will have points which reside in no clusters
# T1 == T2 will cause all points to reside in mutually exclusive clusters
# Distance metric can be any from here: http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.pairwise_distances.html
# filemap may be a list of point names in their order in X. If included, row numbers from X will be replaced with names from filemap. 
 
def main():
    # read in and format data from .csv file
    df = pd.read_csv('/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/CSV_files/10_genome_k4_kmer_top_10_table.csv')
    kmers = list(df['Unnamed: 0'])
    df['kmers'] = kmers
    df.set_index('kmers',inplace = True)
    del df['Unnamed: 0']
    df = df.T
    df.head(3)
    
    # calculate and format distance matrix
    X1_dist = pd.DataFrame(pairwise_distances(df, metric='cosine'))
    X1_dist.columns = df.index
    X1_dist.index = df.index
    
    # distance threshold for clustering proteins together
    threshold = 0.5
    
    # set up and format data structures for algorithm
    canopies = dict()
    elligible_points = list(X1_dist.columns)
    points_in_threshold = []
    clustered_points = []
    
    # set up and format final grouping_list 
    grouping_list = pd.DataFrame(list([None] * X1_dist.shape[0]))
    grouping_list = grouping_list.T
    grouping_list.columns = X1_dist.columns

    iteration = 0
    while len(elligible_points) > 0:
        iteration = iteration + 1
        print("iteration: " + str(iteration))
        # record which organisms' proteins have been clustered into this cluster
        # only one protein per organism is allowed in this cluster
        clustered_organisms = []

        center_point = list(elligible_points)[-1]
        i = len(canopies)
        
        for point in elligible_points:
            if(X1_dist[center_point][point] < threshold):
                # check for organisms that have already been clustered
                if(point.split('_')[0] not in clustered_organisms):
                    points_in_threshold.append(point) 
                    clustered_organisms.append(point.split('_')[0])
                    
        # cluster points which have not already been clustered
        points_to_cluster = set(points_in_threshold).difference(set(clustered_points))
        clustered_points.extend(points_to_cluster)
        elligible_points = set(elligible_points).difference(set(points_to_cluster))
        
        canopies[i] = {"c":iteration, "points": points_to_cluster}
        
        for entry in canopies[i]["points"]:
            grouping_list[entry] = iteration

    with open('/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/CSV_files/10_genome_top_10_4mer_cosine_canopy_clusters_0.5_ps.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in canopies.items():
            writer.writerow([key, value])
    
    with open('/Users/matthewthompson/Documents/UAMS_SURF/K-mer_testing/CSV_files/10_genome_top_10_4mer_grouping_list_0.5_ps.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(list(grouping_list.iloc[0])) 
    
if __name__ == "__main__":
    main()

Once I got this code working, I then got the pangeome results using FindMyFriends and compared the results when paralogues were grouped against when paralogues were separated. My results: ![paraComparison](images/w3/paraloguesComparison.png)

I then investigated the FindMyFriends (1.1.6) code to see how it grouped the gene groups (into Core, Accessory, Singleton), and I found that groups were only classified as Core when the gene group had a gene from every single organism. I then lowered the threshold for Core designation to be 90% for shared samples, and I got a larger core section for the pangenome: ![90core](images/w3/10_genome_top_10_4mers_0.5_ps_0.9_core.png)